In [142]:
#!/usr/bin/python3.3
# -*-coding:Utf-8 -*


import pandas as pd 
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt 
from datetime import datetime
pd.options.mode.chained_assignment = None  # default='warn'
import xgboost as xgb
# reading data
train = pd.read_csv("sources/train.csv")
test = pd.read_csv("sources/test.csv")

In [143]:
#add a new column 'date' convert into datetime for both training and test set
train['date']=0
for i in range(len(train)) :
	train['date'][i]=datetime.strptime(train['datetime'][i],'%Y-%m-%d %H:%M:%S')

train=train.set_index("datetime")


test['date']=0
for i in range(len(test)) :
	test['date'][i]=datetime.strptime(test['datetime'][i],'%Y-%m-%d %H:%M:%S')

test=test.set_index("datetime")

In [144]:
'''
#categorize training set
train = train
train['weather'] = train['weather'].astype('category')
train['holiday'] = train['holiday'].astype('category')
train['workingday'] = train['workingday'].astype('category')
train['season'] = train['season'].astype('category')

#factorize test set
test = test
test['weather'] = test['weather'].astype('category')
test['holiday'] = test['holiday'].astype('category')
test['workingday'] = test['workingday'].astype('category')
test['season'] = test['season'].astype('category')
'''

"\n#categorize training set\ntrain = train\ntrain['weather'] = train['weather'].astype('category')\ntrain['holiday'] = train['holiday'].astype('category')\ntrain['workingday'] = train['workingday'].astype('category')\ntrain['season'] = train['season'].astype('category')\n\n#factorize test set\ntest = test\ntest['weather'] = test['weather'].astype('category')\ntest['holiday'] = test['holiday'].astype('category')\ntest['workingday'] = test['workingday'].astype('category')\ntest['season'] = test['season'].astype('category')\n"

In [145]:
#create day of week column and hour column and a year colum then convert into category for both training and test set
train['day']=0
train['hour']=0
train['year']=0
for i in range(len(train)) :
    train['day'][i] = train['date'][i].weekday()
    train['hour'][i] = train['date'][i].hour
    train['year'][i]=train['date'][i].year

test['day']=0
test['hour']=0
test['year']=0
for i in range(len(test)) :
    test['day'][i] = test['date'][i].weekday()
    test['hour'][i] = test['date'][i].hour
    test['year'][i]=test['date'][i].year


In [146]:
#create ahot () variable and convert to category (4 category) for both training and test set
train['ahot'] = 4
for i in range(len(train)) :
	currentTemp = train['atemp'][i]
	if (currentTemp>=24.24 and currentTemp<31.06 ):
		train['ahot'][i] = 3
	elif (currentTemp>=16.6 and currentTemp<24.24):
		train['ahot'][i] = 2
	elif ( currentTemp<16.6) :
		train['ahot'][i] = 1


test['ahot'] = 4
for i in range(len(test)) :
	currentTemp = test['atemp'][i]
	if (currentTemp>=24.24 and currentTemp<31.06 ):
		test['ahot'][i] = 3
	elif (currentTemp>=16.6 and currentTemp<24.24):
		test['ahot'][i] = 2
	elif (currentTemp<16.6) :
		test['ahot'][i] = 1



In [147]:
#create ahot () variable and convert to category (4 category) for both training and test set
train['hot'] = 4
for i in range(len(train)) :
	currentTemp = train['temp'][i]
	if (currentTemp>=20.5 and currentTemp<26.24 ):
		train['hot'][i] = 3
	elif (currentTemp>=13.94 and currentTemp<20.5):
		train['hot'][i] = 2
	elif ( currentTemp<13.94) :
		train['hot'][i] = 1


test['hot'] = 4
for i in range(len(test)) :
	currentTemp = test['temp'][i]
	if (currentTemp>=20.5 and currentTemp<26.24 ):
		test['hot'][i] = 3
	elif (currentTemp>=13.94 and currentTemp<20.5):
		test['hot'][i] = 2
	elif (currentTemp<13.94) :
		test['hot'][i] = 1



In [148]:
train = train.reset_index()
train = train.drop("datetime",1)
train = train.drop(['date'],1)

test = test.reset_index()
result_reg = test[['datetime']]
result_casual = test[['datetime']]

test = test.drop("datetime",1)
test = test.drop('date',1)

In [149]:
train_registered=train.drop("casual",1)
train_registered=train_registered.drop("count",1)

train_casual=train.drop("registered",1)
train_casual=train_casual.drop("count",1)

In [158]:
train_registered

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,registered,day,hour,year,ahot,hot
0,1,0,0,1,9.84,14.395,81,0.0000,13,5,0,2011,1,1
1,1,0,0,1,9.02,13.635,80,0.0000,32,5,1,2011,1,1
2,1,0,0,1,9.02,13.635,80,0.0000,27,5,2,2011,1,1
3,1,0,0,1,9.84,14.395,75,0.0000,10,5,3,2011,1,1
4,1,0,0,1,9.84,14.395,75,0.0000,1,5,4,2011,1,1
5,1,0,0,2,9.84,12.880,75,6.0032,1,5,5,2011,1,1
6,1,0,0,1,9.02,13.635,80,0.0000,0,5,6,2011,1,1
7,1,0,0,1,8.20,12.880,86,0.0000,2,5,7,2011,1,1
8,1,0,0,1,9.84,14.395,75,0.0000,7,5,8,2011,1,1
9,1,0,0,1,13.12,17.425,76,0.0000,6,5,9,2011,2,1


In [151]:
train_registered.columns

Index(['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp',
       'humidity', 'windspeed', 'registered', 'day', 'hour', 'year', 'ahot',
       'hot'],
      dtype='object')

In [152]:
train_casual.columns

Index(['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp',
       'humidity', 'windspeed', 'casual', 'day', 'hour', 'year', 'ahot',
       'hot'],
      dtype='object')

In [153]:
X_train = train_registered.drop("registered",1)
Y_train = train_registered['registered']
T_train_xgb = xgb.DMatrix(X_train, Y_train)
params = {"objective": "reg:linear", "bst:max_depth":2}
gbm_reg = xgb.train(dtrain=T_train_xgb, params = params)
X_test = xgb.DMatrix(test)
Y_pred = gbm.predict(X_test)
print(Y_pred)
xgb.plot_importance(gbm)


[ 1.21601903  1.27089262  1.30699039 ...,  4.72555733  3.19396782
  2.51288199]


In [154]:
plt.show()

In [155]:
result_reg['count'] = pd.DataFrame(Y_pred)
#result[result['count']<=0] = 0

result_final_reg = result_reg.set_index('datetime')
result_final_reg[result_final_reg['count']<=0]=0
result_final_reg

,count
datetime,
2011-01-20 00:00:00,1.216019
2011-01-20 01:00:00,1.270893
2011-01-20 02:00:00,1.306990
2011-01-20 03:00:00,0.939128
2011-01-20 04:00:00,0.801862
2011-01-20 05:00:00,0.291131
2011-01-20 06:00:00,1.447177
2011-01-20 07:00:00,3.394145
2011-01-20 08:00:00,6.150535


In [156]:
X_train = train_casual.drop("casual",1)
Y_train = train_casual['casual']
T_train_xgb = xgb.DMatrix(X_train, Y_train)
params = {"objective": "reg:linear"}
gbm = xgb.train(dtrain=T_train_xgb, params = params)
X_test = xgb.DMatrix(test)
Y_pred = gbm.predict(X_test)
print(Y_pred)
xgb.plot_importance(gbm)


[ 1.21601903  1.27089262  1.30699039 ...,  4.72555733  3.19396782
  2.51288199]


In [157]:
result_casual['count'] = pd.DataFrame(Y_pred)
#result[result['count']<=0] = 0

result_final_casual = result_casual.set_index('datetime')
result_final_casual[result_final_casual['count']<=0]=0
result_final_casual

,count
datetime,
2011-01-20 00:00:00,1.216019
2011-01-20 01:00:00,1.270893
2011-01-20 02:00:00,1.306990
2011-01-20 03:00:00,0.939128
2011-01-20 04:00:00,0.801862
2011-01-20 05:00:00,0.291131
2011-01-20 06:00:00,1.447177
2011-01-20 07:00:00,3.394145
2011-01-20 08:00:00,6.150535


In [36]:
result_final.to_csv('Résultats/XGBoost/submissionXGBoost.csv')